# Project Title

## Overview

   Our company is making the venture into the airline industry. We have been tasked with determining which aircraft our company should purchase. In this notebook, we have analyzed a large dataset on aircraft accidents in order to provide three recommendations in that vein.

## Business Understanding

   The airline business notoriously walks a fine line between risk and profit. Each company must answer to two main stakeholders: their shareholders and their passengers. The former demands profit maximization and cost minimization, and the latter deserves safe, speedy flights.
   
   We believe that these two goals work hand-in-hand: by providing customers safe, high-quality aircraft to fly in, we reduce the risk of expensive accidents, increasing our profits.

## Data Understanding

The dataset we will be using can be accessed here: [Aviation Accident Database 1948-2022](https://www.kaggle.com/datasets/khsamaha/aviation-accident-database-synopses). It is publicly available through the [kaggle](https://www.kaggle.com/) database. 

This dataset is provided by the National Transportation Safety Board, an independent government body which enforces aeronautics safety guidelines, investigates incidents, advocates for victims, and more.

The NTSB has compiled this cumulative file containing over 90,000 individual records of different types of aircraft accidents from the years 1948-2022. Each row represents one accident and each column (of which there are 31) represents a detail about said accident. 

Our goal in this project is to discover the airplane types with the lowest fatalities and damage, as these will also minimize the cost of doing business. Thus, this dataset is extremely relevant to our business question.

In this section we will explore the full dataset so that we can later decide which specific data is most relevant to our research. 

In [1]:
import pandas as pd
import matplotlib.pyplot as plt

In [7]:
df = pd.read_csv('Final_Data/Aviation_Data.csv', parse_dates=['Event.Date', 'Publication.Date'])
df.head()

/Users/shelleywang/anaconda3/envs/learn-env/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3145: DtypeWarning: Columns (6,7,28) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


,Event.Id,Investigation.Type,Accident.Number,Event.Date,Location,Country,Latitude,Longitude,Airport.Code,Airport.Name,...,Purpose.of.flight,Air.carrier,Total.Fatal.Injuries,Total.Serious.Injuries,Total.Minor.Injuries,Total.Uninjured,Weather.Condition,Broad.phase.of.flight,Report.Status,Publication.Date
0,20001218X45444,Accident,SEA87LA080,1948-10-24,"MOOSE CREEK, ID",United States,NaN,NaN,NaN,NaN,...,Personal,NaN,2.0,0.0,0.0,0.0,UNK,Cruise,Probable Cause,NaT
1,20001218X45447,Accident,LAX94LA336,1962-07-19,"BRIDGEPORT, CA",United States,NaN,NaN,NaN,NaN,...,Personal,NaN,4.0,0.0,0.0,0.0,UNK,Unknown,Probable Cause,1996-09-19
2,20061025X01555,Accident,NYC07LA005,1974-08-30,"Saltville, VA",United States,36.9222,-81.8781,NaN,NaN,...,Personal,NaN,3.0,NaN,NaN,NaN,IMC,Cruise,Probable Cause,2007-02-26
3,20001218X45448,Accident,LAX96LA321,1977-06-19,"EUREKA, CA",United States,NaN,NaN,NaN,NaN,...,Personal,NaN,2.0,0.0,0.0,0.0,IMC,Cruise,Probable Cause,2000-12-09
4,20041105X01764,Accident,CHI79FA064,1979-08-02,"Canton, OH",United States,NaN,NaN,NaN,NaN,...,Personal,NaN,1.0,2.0,NaN,0.0,VMC,Approach,Probable Cause,1980-04-16


In [8]:
df.head()

,Event.Id,Investigation.Type,Accident.Number,Event.Date,Location,Country,Latitude,Longitude,Airport.Code,Airport.Name,...,Purpose.of.flight,Air.carrier,Total.Fatal.Injuries,Total.Serious.Injuries,Total.Minor.Injuries,Total.Uninjured,Weather.Condition,Broad.phase.of.flight,Report.Status,Publication.Date
0,20001218X45444,Accident,SEA87LA080,1948-10-24,"MOOSE CREEK, ID",United States,NaN,NaN,NaN,NaN,...,Personal,NaN,2.0,0.0,0.0,0.0,UNK,Cruise,Probable Cause,NaT
1,20001218X45447,Accident,LAX94LA336,1962-07-19,"BRIDGEPORT, CA",United States,NaN,NaN,NaN,NaN,...,Personal,NaN,4.0,0.0,0.0,0.0,UNK,Unknown,Probable Cause,1996-09-19
2,20061025X01555,Accident,NYC07LA005,1974-08-30,"Saltville, VA",United States,36.9222,-81.8781,NaN,NaN,...,Personal,NaN,3.0,NaN,NaN,NaN,IMC,Cruise,Probable Cause,2007-02-26
3,20001218X45448,Accident,LAX96LA321,1977-06-19,"EUREKA, CA",United States,NaN,NaN,NaN,NaN,...,Personal,NaN,2.0,0.0,0.0,0.0,IMC,Cruise,Probable Cause,2000-12-09
4,20041105X01764,Accident,CHI79FA064,1979-08-02,"Canton, OH",United States,NaN,NaN,NaN,NaN,...,Personal,NaN,1.0,2.0,NaN,0.0,VMC,Approach,Probable Cause,1980-04-16


In [9]:
df.tail()

,Event.Id,Investigation.Type,Accident.Number,Event.Date,Location,Country,Latitude,Longitude,Airport.Code,Airport.Name,...,Purpose.of.flight,Air.carrier,Total.Fatal.Injuries,Total.Serious.Injuries,Total.Minor.Injuries,Total.Uninjured,Weather.Condition,Broad.phase.of.flight,Report.Status,Publication.Date
90343,20221227106491,Accident,ERA23LA093,2022-12-26,"Annapolis, MD",United States,NaN,NaN,NaN,NaN,...,Personal,NaN,0.0,1.0,0.0,0.0,NaN,NaN,NaN,2022-12-29
90344,20221227106494,Accident,ERA23LA095,2022-12-26,"Hampton, NH",United States,NaN,NaN,NaN,NaN,...,NaN,NaN,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaT
90345,20221227106497,Accident,WPR23LA075,2022-12-26,"Payson, AZ",United States,341525N,1112021W,PAN,PAYSON,...,Personal,NaN,0.0,0.0,0.0,1.0,VMC,NaN,NaN,2022-12-27
90346,20221227106498,Accident,WPR23LA076,2022-12-26,"Morgan, UT",United States,NaN,NaN,NaN,NaN,...,Personal,MC CESSNA 210N LLC,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaT
90347,20221230106513,Accident,ERA23LA097,2022-12-29,"Athens, GA",United States,NaN,NaN,NaN,NaN,...,Personal,NaN,0.0,1.0,0.0,1.0,NaN,NaN,NaN,2022-12-30


In [10]:
df.shape

(90348, 31)

In [11]:
df.describe(datetime_is_numeric=True)

,Event.Date,Number.of.Engines,Total.Fatal.Injuries,Total.Serious.Injuries,Total.Minor.Injuries,Total.Uninjured,Publication.Date
count,88889,82805.000000,77488.000000,76379.000000,76956.000000,82977.000000,73659
mean,1999-09-17 17:13:39.354475904,1.146585,0.647855,0.279881,0.357061,5.325440,2004-05-01 19:38:20.932676224
min,1948-10-24 00:00:00,0.000000,0.000000,0.000000,0.000000,0.000000,1980-04-16 00:00:00
25%,1989-01-15 00:00:00,1.000000,0.000000,0.000000,0.000000,0.000000,1993-09-14 00:00:00
50%,1998-07-18 00:00:00,1.000000,0.000000,0.000000,0.000000,1.000000,2001-10-07 00:00:00
75%,2009-07-01 00:00:00,1.000000,0.000000,0.000000,0.000000,2.000000,2020-09-25 00:00:00
max,2022-12-29 00:00:00,8.000000,349.000000,161.000000,380.000000,699.000000,2022-12-30 00:00:00
std,NaN,0.446510,5.485960,1.544084,2.235625,27.913634,NaN


In [12]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 90348 entries, 0 to 90347
Data columns (total 31 columns):
 #   Column                  Non-Null Count  Dtype         
---  ------                  --------------  -----         
 0   Event.Id                88889 non-null  object        
 1   Investigation.Type      90348 non-null  object        
 2   Accident.Number         88889 non-null  object        
 3   Event.Date              88889 non-null  datetime64[ns]
 4   Location                88837 non-null  object        
 5   Country                 88663 non-null  object        
 6   Latitude                34382 non-null  object        
 7   Longitude               34373 non-null  object        
 8   Airport.Code            50249 non-null  object        
 9   Airport.Name            52790 non-null  object        
 10  Injury.Severity         87889 non-null  object        
 11  Aircraft.damage         85695 non-null  object        
 12  Aircraft.Category       32287 non-null  object

In [13]:
df.isna().sum()

Event.Id                   1459
Investigation.Type            0
Accident.Number            1459
Event.Date                 1459
Location                   1511
Country                    1685
Latitude                  55966
Longitude                 55975
Airport.Code              40099
Airport.Name              37558
Injury.Severity            2459
Aircraft.damage            4653
Aircraft.Category         58061
Registration.Number        2776
Make                       1522
Model                      1551
Amateur.Built              1561
Number.of.Engines          7543
Engine.Type                8536
FAR.Description           58325
Schedule                  77766
Purpose.of.flight          7651
Air.carrier               73700
Total.Fatal.Injuries      12860
Total.Serious.Injuries    13969
Total.Minor.Injuries      13392
Total.Uninjured            7371
Weather.Condition          5951
Broad.phase.of.flight     28624
Report.Status              7840
Publication.Date          16689
dtype: i

Below, we explore columns that we may want to drop. Taking a value count shows us the unique data points in that column, elucidating what that column is describing.

In [14]:
df['Air.carrier'].value_counts()

Pilot                      258
American Airlines           90
United Airlines             89
Delta Air Lines             53
SOUTHWEST AIRLINES CO       42
                          ... 
Pawan Hans Limited           1
Blue Mountain Aviation       1
MCCALL AVIATION INC          1
Skywest Airlines Inc         1
PRO AGRI SPRAYING, Inc.      1
Name: Air.carrier, Length: 13590, dtype: int64

In [15]:
df['Purpose.of.flight'].value_counts()

Personal                     49448
Instructional                10601
Unknown                       6802
Aerial Application            4712
Business                      4018
Positioning                   1646
Other Work Use                1264
Ferry                          812
Aerial Observation             794
Public Aircraft                720
Executive/corporate            553
Flight Test                    405
Skydiving                      182
External Load                  123
Public Aircraft - Federal      105
Banner Tow                     101
Air Race show                   99
Public Aircraft - Local         74
Public Aircraft - State         64
Air Race/show                   59
Glider Tow                      53
Firefighting                    40
Air Drop                        11
ASHO                             6
PUBS                             4
PUBL                             1
Name: Purpose.of.flight, dtype: int64

In [16]:
df['FAR.Description'].value_counts()

091                               18221
Part 91: General Aviation          6486
NUSN                               1584
NUSC                               1013
137                                1010
135                                 746
121                                 679
Part 137: Agricultural              437
UNK                                 371
Part 135: Air Taxi & Commuter       298
PUBU                                253
129                                 246
Part 121: Air Carrier               165
133                                 107
Part 129: Foreign                   100
Non-U.S., Non-Commercial             97
Non-U.S., Commercial                 93
Part 133: Rotorcraft Ext. Load       32
Unknown                              22
Public Use                           19
091K                                 14
ARMF                                  8
125                                   5
Part 125: 20+ Pax,6000+ lbs           5
107                                   4


In [17]:
df['Schedule'].value_counts()

NSCH    4474
UNK     4099
SCHD    4009
Name: Schedule, dtype: int64

In [18]:
df['Report.Status'].value_counts()

Probable Cause                                                                                                                                                                                                                              61754
Foreign                                                                                                                                                                                                                                      1999
<br /><br />                                                                                                                                                                                                                                  167
Factual                                                                                                                                                                                                                                       145
The pilot's failure to maintain 

## Data Preparation

Now that we've explored the full dataset, we will remove data that is unecessary to our search for the best airplanes. In other words, we will shave down the NTSB file and save that new chunk of data for our later analysis. 

Explaining columns which might be relevant to other business questions: airport.code, airport.name, FAR Description, schedule, purpose.of.flight, air.carrier, broad.phase.of.flight

### Making a Subset by Dropping Columns and Rows

First we will define our subset as a colleciton of relevant columns from the full NTSB dataset. Below you will see that we have dropped 16 of the original 31 columns. 

We dropped 'Event.Id', 'Accident.Number', and 'Registration.Number" because these are unique identifying number-letter strings whose use as identification is redundant. We have an Index column where each row is numbered. This makes these two columns obsolete.

We dropped 'Investigation Type', 'Report.Status', and 'Publication.Date' because these details are only relevant to the procedural investigation of each accident, not the aircraft qualities.

We dropped 'Latitude', 'Longitude', 'Airport.Code', and 'Airport.Name' because we will be keeping the 'Location' and 'Country' columns as our chosen geographical data. These dropped columns are highly specific and therefore less workable for our purposes. 

The 'FAR.Description', 'Schedule', and 'Purpose.of.flight' columns provide data regarding the type of each trip. FAR description identifies the rulebook under which the pilot flew, Schedule ident
ifies 

Explaining which columns we chose to drop b/c obviously not relevant or redundant: event.id, ivestigation.type, accident.number. latitude, longitude, registration.number, report.status, publication.date

In [19]:
df_subset = df[['Event.Date', 'Location', 'Country', 'Number.of.Engines',
                'Aircraft.damage', 'Aircraft.Category', 'Make', 'Model', 
                'Amateur.Built', 'Engine.Type',
                'Total.Fatal.Injuries', 'Total.Minor.Injuries', 
                'Total.Uninjured', 'Total.Serious.Injuries', 
                'Weather.Condition']]


In [20]:
# Standardizing the formatting of the column names

df_subset = df_subset.rename(columns = lambda x: x.lower())

In [21]:
# Keeping only airplanes

df_subset = df_subset.loc[df_subset['aircraft.category'] == 'Airplane']

In [22]:
# Dropping the 'aircraft.category' column

df_subset = df_subset.drop(columns=['aircraft.category'])

In [23]:
# Keeping only non-amateur built airplanes

df_subset = df_subset.loc[df_subset['amateur.built'] == 'No']

In [24]:
# Dropping the 'amateur.built' column

df_subset = df_subset.drop(columns=['amateur.built'])

In [25]:
# Keeping only the year (first 4 characters) from the 'event.date' column

df_subset = df_subset.rename(columns = lambda x: x.lower())
df_subset['event.date'] = pd.to_datetime(df_subset['event.date']).dt.year


In [26]:
# Keeping only the state abbreviations from 'location' column into a new column

df_subset['state'] = df_subset['location'].str.split(",").str[1]


In [27]:
# Dropping the 'location' column

df_subset = df_subset.drop(columns=['location'])

In [28]:
# Exploring the 'weather.condition' column

df_subset['weather.condition'].value_counts()

VMC    19711
IMC     1363
Unk      215
UNK      149
Name: weather.condition, dtype: int64

In [29]:
# Cleaning typos from weather.condition

replace_dict = {'Unk':'UNK'}
df_subset['weather.condition'] = df_subset['weather.condition'].replace(replace_dict)
df_subset['weather.condition'].value_counts()

VMC    19711
IMC     1363
UNK      364
Name: weather.condition, dtype: int64

In [32]:
# Exploring the 'engine.type' column

df_subset['engine.type'].value_counts()

Reciprocating      17846
Turbo Prop          1332
Turbo Fan            967
Turbo Jet            153
Unknown              135
Geared Turbofan       12
Turbo Shaft           11
Electric               5
Name: engine.type, dtype: int64

In [33]:
# Cleaning typos from engine.type

replace_dict3 = {'UNK':'Unknown'}
df_subset['engine.type'] = df_subset['engine.type'].replace(replace_dict3)
df_subset['engine.type'].value_counts()

Reciprocating      17846
Turbo Prop          1332
Turbo Fan            967
Turbo Jet            153
Unknown              135
Geared Turbofan       12
Turbo Shaft           11
Electric               5
Name: engine.type, dtype: int64

Understanding for our new data set having dropped unecessary info and cleaned redundant values

In [34]:
df_subset.head()

,event.date,country,number.of.engines,aircraft.damage,make,model,engine.type,total.fatal.injuries,total.minor.injuries,total.uninjured,total.serious.injuries,weather.condition,state
5,1979,United States,2.0,Substantial,Mcdonnell Douglas,DC9,Turbo Fan,NaN,1.0,44.0,NaN,VMC,MA
7,1982,United States,1.0,Substantial,Cessna,140,Reciprocating,0.0,0.0,2.0,0.0,VMC,WA
8,1982,United States,2.0,Substantial,Cessna,401B,Reciprocating,0.0,0.0,2.0,0.0,IMC,NJ
12,1982,United States,1.0,Destroyed,Bellanca,17-30A,Reciprocating,0.0,1.0,0.0,0.0,IMC,LA
13,1982,United States,1.0,Destroyed,Cessna,R172K,Reciprocating,1.0,0.0,0.0,0.0,IMC,TX


In [35]:
df_subset.shape

(24417, 13)

In [36]:
df_subset.describe()

,event.date,number.of.engines,total.fatal.injuries,total.minor.injuries,total.uninjured,total.serious.injuries
count,24417.000000,21866.000000,21651.000000,21855.000000,23699.000000,21571.000000
mean,2009.465332,1.178542,0.692301,0.237795,7.072071,0.304112
std,11.509243,0.428836,6.305217,1.689566,33.701013,2.222186
min,1979.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,2008.000000,1.000000,0.000000,0.000000,0.000000,0.000000
50%,2012.000000,1.000000,0.000000,0.000000,1.000000,0.000000
75%,2017.000000,1.000000,0.000000,0.000000,2.000000,0.000000
max,2022.000000,8.000000,295.000000,200.000000,588.000000,161.000000


In [37]:
df_subset.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 24417 entries, 5 to 90345
Data columns (total 13 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   event.date              24417 non-null  int64  
 1   country                 24410 non-null  object 
 2   number.of.engines       21866 non-null  float64
 3   aircraft.damage         23147 non-null  object 
 4   make                    24414 non-null  object 
 5   model                   24399 non-null  object 
 6   engine.type             20461 non-null  object 
 7   total.fatal.injuries    21651 non-null  float64
 8   total.minor.injuries    21855 non-null  float64
 9   total.uninjured         23699 non-null  float64
 10  total.serious.injuries  21571 non-null  float64
 11  weather.condition       21438 non-null  object 
 12  state                   24393 non-null  object 
dtypes: float64(5), int64(1), object(7)
memory usage: 2.6+ MB


### Cleaning Null Values

In [38]:
df_subset.isna().sum()

event.date                   0
country                      7
number.of.engines         2551
aircraft.damage           1270
make                         3
model                       18
engine.type               3956
total.fatal.injuries      2766
total.minor.injuries      2562
total.uninjured            718
total.serious.injuries    2846
weather.condition         2979
state                       24
dtype: int64

In [39]:
# Dropping rows in columns that have very few nulls

df_subset = df_subset.dropna(subset=['country', 'make', 'model', 'state'])
df_subset.isna().sum()

event.date                   0
country                      0
number.of.engines         2540
aircraft.damage           1263
make                         0
model                        0
engine.type               3942
total.fatal.injuries      2752
total.minor.injuries      2545
total.uninjured            709
total.serious.injuries    2828
weather.condition         2968
state                        0
dtype: int64

In [40]:
# Filling the NaN's in the 4 injuries columns with 0's

df_subset['total.serious.injuries'].fillna(0, inplace=True)
df_subset['total.fatal.injuries'].fillna(0, inplace=True)
df_subset['total.minor.injuries'].fillna(0, inplace=True)
df_subset['total.uninjured'].fillna(0, inplace=True)

df_subset.isna().sum()

event.date                   0
country                      0
number.of.engines         2540
aircraft.damage           1263
make                         0
model                        0
engine.type               3942
total.fatal.injuries         0
total.minor.injuries         0
total.uninjured              0
total.serious.injuries       0
weather.condition         2968
state                        0
dtype: int64

In [41]:
# Replacing nulls in the 'aircraft.damage' column

df_subset['aircraft.damage'].value_counts()
df_subset['aircraft.damage'] = df_subset['aircraft.damage'].fillna('N/A')

In [42]:
# Combining 'Unknown' and 'N/A' values in in the 'aircraft.damage' column

replace_dict1 = {'Unknown':'N/A'}
df_subset['aircraft.damage'] = df_subset['aircraft.damage'].replace(replace_dict1)
df_subset['aircraft.damage'].value_counts()

Substantial    18965
Destroyed       3129
N/A             1360
Minor            920
Name: aircraft.damage, dtype: int64

In [43]:
df_subset.isna().sum()

event.date                   0
country                      0
number.of.engines         2540
aircraft.damage              0
make                         0
model                        0
engine.type               3942
total.fatal.injuries         0
total.minor.injuries         0
total.uninjured              0
total.serious.injuries       0
weather.condition         2968
state                        0
dtype: int64

In [44]:
# Replacing nulls in the 'engine.type' column

df_subset['engine.type'].value_counts()
df_subset['engine.type'] = df_subset['engine.type'].fillna('Unknown')

In [45]:
# Combining 'Unknown' and 'N/A' values in in the 'engine.type' column

replace_dict2 = {'Unknown':'N/A'}
df_subset['engine.type'] = df_subset['engine.type'].replace(replace_dict2)
df_subset['engine.type'].value_counts()

Reciprocating      17832
N/A                 4070
Turbo Prop          1330
Turbo Fan            961
Turbo Jet            153
Geared Turbofan       12
Turbo Shaft           11
Electric               5
Name: engine.type, dtype: int64

In [46]:
df_subset.isna().sum()

event.date                   0
country                      0
number.of.engines         2540
aircraft.damage              0
make                         0
model                        0
engine.type                  0
total.fatal.injuries         0
total.minor.injuries         0
total.uninjured              0
total.serious.injuries       0
weather.condition         2968
state                        0
dtype: int64

In [47]:
# Replacing nulls in the 'weather.condition' column

df_subset['weather.condition'].value_counts()
df_subset['weather.condition'] = df_subset['weather.condition'].fillna('N/A')

In [48]:
df_subset.isna().sum()

event.date                   0
country                      0
number.of.engines         2540
aircraft.damage              0
make                         0
model                        0
engine.type                  0
total.fatal.injuries         0
total.minor.injuries         0
total.uninjured              0
total.serious.injuries       0
weather.condition            0
state                        0
dtype: int64

Now, to deal with nulls in the 'number.of.engines' column, we will search for an appropriate measure of central tendency. We cannot fill the nulls with 0's, as this doesn't make conceptual sense: there are no planes with no engines.

In [49]:
# Checking the distribution of the 'number.of.engines' column

df_subset['number.of.engines'].value_counts()

1.0    18200
2.0     3466
4.0      101
3.0       56
0.0        9
6.0        1
8.0        1
Name: number.of.engines, dtype: int64

In [50]:
number.engines.hist, ax = plt.subplots
ax.hist(df_subset['number.of.engines'], bins = 7)
ax.set_xlabel('number.of.engines')
ax.set_ylabel('count')
ax.set_title('Distribution of number.of.engines')

TypeError: cannot unpack non-iterable function object

In [51]:
# Replacing nulls in the 'number.of.engines' column

df_subset['number.of.engines'].median()
df_subset['number.of.engines'] = df_subset['number.of.engines'].fillna(df_subset['number.of.engines'].median())

### Consolidating the serious injuries and minor injuries columns

why we do this

In [52]:
df_subset['total.nonfatal.injuries'] = df_subset['total.minor.injuries'] + df_subset['total.serious.injuries']

In [53]:
df_subset = df_subset.drop(columns=['total.serious.injuries', 'total.minor.injuries'])

### Typecasting: Changing float columns into integer columns 

Why we do this

In [54]:
df_subset.dtypes

event.date                   int64
country                     object
number.of.engines          float64
aircraft.damage             object
make                        object
model                       object
engine.type                 object
total.fatal.injuries       float64
total.uninjured            float64
weather.condition           object
state                       object
total.nonfatal.injuries    float64
dtype: object

In [55]:
df_subset['number.of.engines'] = df_subset['number.of.engines'].astype(int)

In [58]:
df_subset['total.nonfatal.injuries'] = df_subset['total.nonfatal.injuries'].astype(int)
df_subset['total.fatal.injuries'] = df_subset['total.fatal.injuries'].astype(int)
df_subset['total.uninjured'] = df_subset['total.uninjured'].astype(int)

In [59]:
df_subset.dtypes

event.date                  int64
country                    object
number.of.engines           int64
aircraft.damage            object
make                       object
model                      object
engine.type                object
total.fatal.injuries        int64
total.uninjured             int64
weather.condition          object
state                      object
total.nonfatal.injuries     int64
dtype: object

### Calculating/creating new injuries percentage columns

In [60]:
# Turning injury columns into percentage of total passengers
df_subset['fatal.injuries.perc'] = round(((df_subset['total.fatal.injuries'] / (df_subset['total.fatal.injuries'] + 
                                                                             df_subset['total.nonfatal.injuries'] + 
                                                                             df_subset['total.uninjured'])) * 100), 1)
df_subset['nonfatal.injuries.perc'] = round(((df_subset['total.nonfatal.injuries'] / (df_subset['total.fatal.injuries'] + 
                                                                             df_subset['total.nonfatal.injuries'] + 
                                                                             df_subset['total.uninjured'])) * 100), 1)
df_subset['uninjured.perc'] = round(((df_subset['total.uninjured'] / (df_subset['total.fatal.injuries'] + 
                                                                             df_subset['total.nonfatal.injuries'] + 
                                                                             df_subset['total.uninjured'])) * 100), 1)

In [61]:
# Cleaning resulting nulls

df_subset.isna().sum()

df_subset['fatal.injuries.perc'].fillna(0, inplace=True)
df_subset['nonfatal.injuries.perc'].fillna(0, inplace=True)
df_subset['uninjured.perc'].fillna(0, inplace=True)

In [62]:
df_subset.isna().sum()

event.date                 0
country                    0
number.of.engines          0
aircraft.damage            0
make                       0
model                      0
engine.type                0
total.fatal.injuries       0
total.uninjured            0
weather.condition          0
state                      0
total.nonfatal.injuries    0
fatal.injuries.perc        0
nonfatal.injuries.perc     0
uninjured.perc             0
dtype: int64

In [64]:
# Checking data type of the new calculated columns

df_subset.dtypes

event.date                  int64
country                    object
number.of.engines           int64
aircraft.damage            object
make                       object
model                      object
engine.type                object
total.fatal.injuries        int64
total.uninjured             int64
weather.condition          object
state                      object
total.nonfatal.injuries     int64
fatal.injuries.perc         int64
nonfatal.injuries.perc      int64
uninjured.perc              int64
dtype: object

In [63]:
# Turning the new injuries percentage columns into integers

df_subset['fatal.injuries.perc'] = df_subset['fatal.injuries.perc'].astype(int)
df_subset['nonfatal.injuries.perc'] = df_subset['nonfatal.injuries.perc'].astype(int)
df_subset['uninjured.perc'] = df_subset['uninjured.perc'].astype(int)

df_subset.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 24374 entries, 5 to 90345
Data columns (total 15 columns):
 #   Column                   Non-Null Count  Dtype 
---  ------                   --------------  ----- 
 0   event.date               24374 non-null  int64 
 1   country                  24374 non-null  object
 2   number.of.engines        24374 non-null  int64 
 3   aircraft.damage          24374 non-null  object
 4   make                     24374 non-null  object
 5   model                    24374 non-null  object
 6   engine.type              24374 non-null  object
 7   total.fatal.injuries     24374 non-null  int64 
 8   total.uninjured          24374 non-null  int64 
 9   weather.condition        24374 non-null  object
 10  state                    24374 non-null  object
 11  total.nonfatal.injuries  24374 non-null  int64 
 12  fatal.injuries.perc      24374 non-null  int64 
 13  nonfatal.injuries.perc   24374 non-null  int64 
 14  uninjured.perc           24374 non-nul

### Translating the aircraft damage column into a numerical scale

why we do this

In [65]:
df_subset['aircraft.damage'].value_counts()

Substantial    18965
Destroyed       3129
N/A             1360
Minor            920
Name: aircraft.damage, dtype: int64

In [66]:
def aircraft_damage_numbers(y):
    if y == "Substantial":
        return 3
    elif y == "Destroyed":
        return 2
    elif y == "Minor":
        return 1
    else:
        return 0

In [67]:
df_subset['aircraft.damage.scale'] = df_subset['aircraft.damage'].map(aircraft_damage_numbers)
df_subset['aircraft.damage.scale'].value_counts()

3    18965
2     3129
0     1360
1      920
Name: aircraft.damage.scale, dtype: int64

### Cleaning the 'make' column

In [68]:
df_subset['make'].head()

5     Mcdonnell Douglas
7                Cessna
8                Cessna
12             Bellanca
13               Cessna
Name: make, dtype: object

In [69]:
# Standardizing the capitalization of every value

df_subset['make'] = df_subset['make'].astype(str).str.lower()
pd.set_option('display.max_rows', None)
df_subset['make'].value_counts().head(50)

cessna                            8431
piper                             4695
beech                             1683
boeing                            1304
mooney                             416
bellanca                           281
grumman                            249
airbus                             243
maule                              232
aeronca                            227
air tractor                        224
cirrus design corp                 220
air tractor inc                    219
champion                           169
luscombe                           163
embraer                            153
stinson                            145
cirrus                             137
north american                     118
mcdonnell douglas                  112
taylorcraft                        110
aero commander                     106
dehavilland                         95
de havilland                        87
aviat aircraft inc                  76
socata                   

In [70]:
# Defining a function to fix the typos in the most common makes

def typos(x):
    if "air tractor" in x:
        return "air tractor"
    elif "cessna" in x:
        return "cessna"
    elif "piper" in x:
        return "piper"
    elif "beech" in x:
        return "beech"
    elif "boeing" in x:
        return "boeing"
    elif "mooney" in x:
        return "mooney"
    elif "grumman" in x:
        return "grumman"
    elif "airbus" in x:
        return "airbus"
    elif "aeronca" in x:
        return "aeronca"
    elif "cirrus" in x:
        return "cirrus"
    elif "champion" in x:
        return "american champion"
    elif "embraer" in x:
        return "embraer"
    elif "havilland" in x:
        return "dehavilland"
    elif "aviat" in x:
        return "aviat"
    elif "diamond" in x:
        return "diamond"
    elif "ercoupe" in x:
        return "ercoupe"
    else:
        return x

In [71]:
df_subset['make'] = df_subset['make'].map(typos)

### Defining and saving our cleaned subset

We are dropping columns used for calculations of percentages and aircraft damage scale

In [72]:
df_subset = df_subset[['event.date', 'country', 'state', 'number.of.engines',
                'make', 'model', 'engine.type',  
                 'fatal.injuries.perc', 'nonfatal.injuries.perc', 'uninjured.perc', 
                'weather.condition', 'aircraft.damage.scale']]

In [73]:
df_subset.head()

,event.date,country,state,number.of.engines,make,model,engine.type,fatal.injuries.perc,nonfatal.injuries.perc,uninjured.perc,weather.condition,aircraft.damage.scale
5,1979,United States,MA,2,mcdonnell douglas,DC9,Turbo Fan,0,2,97,VMC,3
7,1982,United States,WA,1,cessna,140,Reciprocating,0,0,100,VMC,3
8,1982,United States,NJ,2,cessna,401B,Reciprocating,0,0,100,IMC,3
12,1982,United States,LA,1,bellanca,17-30A,Reciprocating,0,100,0,IMC,2
13,1982,United States,TX,1,cessna,R172K,Reciprocating,100,0,0,IMC,2


In [74]:
df_subset.shape

(24374, 12)

In [75]:
df_subset.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 24374 entries, 5 to 90345
Data columns (total 12 columns):
 #   Column                  Non-Null Count  Dtype 
---  ------                  --------------  ----- 
 0   event.date              24374 non-null  int64 
 1   country                 24374 non-null  object
 2   state                   24374 non-null  object
 3   number.of.engines       24374 non-null  int64 
 4   make                    24374 non-null  object
 5   model                   24374 non-null  object
 6   engine.type             24374 non-null  object
 7   fatal.injuries.perc     24374 non-null  int64 
 8   nonfatal.injuries.perc  24374 non-null  int64 
 9   uninjured.perc          24374 non-null  int64 
 10  weather.condition       24374 non-null  object
 11  aircraft.damage.scale   24374 non-null  int64 
dtypes: int64(6), object(6)
memory usage: 2.4+ MB


In [76]:
df_subset.describe()

,event.date,number.of.engines,fatal.injuries.perc,nonfatal.injuries.perc,uninjured.perc,aircraft.damage.scale
count,24374.000000,24374.000000,24374.000000,24374.000000,24374.000000,24374.000000
mean,2009.485230,1.159350,16.399852,19.074957,60.801100,2.628744
std,11.499791,0.408294,36.034714,37.207320,47.748262,0.802325
min,1979.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,2008.000000,1.000000,0.000000,0.000000,0.000000,3.000000
50%,2012.000000,1.000000,0.000000,0.000000,100.000000,3.000000
75%,2017.000000,1.000000,0.000000,0.000000,100.000000,3.000000
max,2022.000000,8.000000,100.000000,100.000000,100.000000,3.000000


In [77]:
df_subset.to_csv('Final_Data/subset.csv')

why we aren't re-indexing the subset

### Defining a second subset that only contains plane makes (brands) with over 100 entries

In [78]:
make_value_counts = df_subset['make'].value_counts()

In [79]:
make_over_100 = make_value_counts.loc[make_value_counts > 100]

In [80]:
make_over_100 = list(make_over_100.index)

In [81]:
make_over_100

['cessna',
 'piper',
 'beech',
 'boeing',
 'mooney',
 'air tractor',
 'grumman',
 'cirrus',
 'airbus',
 'bellanca',
 'aviat',
 'american champion',
 'maule',
 'aeronca',
 'dehavilland',
 'embraer',
 'luscombe',
 'diamond',
 'stinson',
 'north american',
 'mcdonnell douglas',
 'taylorcraft',
 'aero commander']

In [82]:
df_subset_makes = df_subset.loc[df_subset['make'].isin(make_over_100)]
df_subset_makes.head()

,event.date,country,state,number.of.engines,make,model,engine.type,fatal.injuries.perc,nonfatal.injuries.perc,uninjured.perc,weather.condition,aircraft.damage.scale
5,1979,United States,MA,2,mcdonnell douglas,DC9,Turbo Fan,0,2,97,VMC,3
7,1982,United States,WA,1,cessna,140,Reciprocating,0,0,100,VMC,3
8,1982,United States,NJ,2,cessna,401B,Reciprocating,0,0,100,IMC,3
12,1982,United States,LA,1,bellanca,17-30A,Reciprocating,0,100,0,IMC,2
13,1982,United States,TX,1,cessna,R172K,Reciprocating,100,0,0,IMC,2


In [83]:
df_subset_makes.to_csv('Final_Data/df_subset_makes.csv')

In [84]:
df_subset_makes.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 20879 entries, 5 to 90345
Data columns (total 12 columns):
 #   Column                  Non-Null Count  Dtype 
---  ------                  --------------  ----- 
 0   event.date              20879 non-null  int64 
 1   country                 20879 non-null  object
 2   state                   20879 non-null  object
 3   number.of.engines       20879 non-null  int64 
 4   make                    20879 non-null  object
 5   model                   20879 non-null  object
 6   engine.type             20879 non-null  object
 7   fatal.injuries.perc     20879 non-null  int64 
 8   nonfatal.injuries.perc  20879 non-null  int64 
 9   uninjured.perc          20879 non-null  int64 
 10  weather.condition       20879 non-null  object
 11  aircraft.damage.scale   20879 non-null  int64 
dtypes: int64(6), object(6)
memory usage: 2.1+ MB


In [85]:
df_subset_makes.describe()

,event.date,number.of.engines,fatal.injuries.perc,nonfatal.injuries.perc,uninjured.perc,aircraft.damage.scale
count,20879.000000,20879.000000,20879.000000,20879.000000,20879.000000,20879.000000
mean,2009.155228,1.150774,16.018727,18.694957,61.537430,2.629436
std,11.748991,0.391705,35.668712,36.835568,47.534727,0.805133
min,1979.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,2007.000000,1.000000,0.000000,0.000000,0.000000,3.000000
50%,2012.000000,1.000000,0.000000,0.000000,100.000000,3.000000
75%,2017.000000,1.000000,0.000000,0.000000,100.000000,3.000000
max,2022.000000,4.000000,100.000000,100.000000,100.000000,3.000000


Conclusion: we now have two data subsets. Explain them

# Analysis and Results

the groupby's and the visuals produced in tableau embedded following those cells

In [86]:
df_subset.groupby('event.date').mean()

,number.of.engines,fatal.injuries.perc,nonfatal.injuries.perc,uninjured.perc,aircraft.damage.scale
event.date,,,,,
1979,2.000000,0.000000,2.000000,97.000000,3.000000
1982,1.177883,15.606696,19.960433,64.401082,2.606358
1983,2.200000,11.500000,5.700000,82.600000,1.400000
1984,1.714286,28.571429,0.000000,71.428571,2.428571
1985,1.333333,19.333333,47.166667,33.333333,2.333333
1986,1.375000,49.000000,13.250000,37.500000,1.625000
1987,2.000000,50.000000,0.500000,49.250000,1.750000
1988,1.583333,25.000000,16.833333,57.916667,1.750000
1989,1.000000,47.571429,33.285714,19.000000,2.571429


In [87]:
df_subset.groupby('country').mean()

,event.date,number.of.engines,fatal.injuries.perc,nonfatal.injuries.perc,uninjured.perc,aircraft.damage.scale
country,,,,,,
Afghanistan,2012.500000,1.700000,20.000000,0.900000,38.900000,2.500000
Albania,2019.000000,1.000000,0.000000,0.000000,0.000000,0.000000
American Samoa,2022.000000,1.066667,33.333333,20.000000,20.000000,1.533333
Angola,2005.000000,1.250000,25.000000,0.000000,50.000000,2.250000
Antarctica,2011.200000,1.600000,20.000000,2.200000,57.600000,1.800000
Antigua and Barbuda,2019.000000,1.000000,0.000000,0.000000,0.000000,3.000000
Argentina,2014.913793,1.155172,29.310345,1.913793,49.724138,1.706897
Australia,2014.583893,1.255034,44.785235,4.644295,21.677852,1.523490
Austria,2013.590909,1.181818,51.863636,9.454545,20.454545,2.181818


In [88]:
df_subset.groupby('state').mean()

,event.date,number.of.engines,fatal.injuries.perc,nonfatal.injuries.perc,uninjured.perc,aircraft.damage.scale
state,,,,,,
,2013.565217,1.456522,20.239130,13.500000,55.282609,2.195652
,2022.000000,1.127907,21.168605,7.761628,36.738372,1.587209
AK,2009.746541,1.075472,7.474843,14.151572,78.215094,2.893711
AL,2007.503311,1.158940,16.801325,21.682119,61.493377,2.788079
ANTARCTICA,2007.000000,2.000000,0.000000,10.000000,90.000000,3.000000
AO,2015.062500,1.312500,59.375000,9.375000,31.125000,2.312500
AR,2007.669333,1.077333,15.269333,24.290667,60.168000,2.802667
ARGENTINA,2002.000000,1.000000,0.000000,1.000000,98.666667,3.000000
AZ,2008.621656,1.132484,11.914650,15.806369,71.363057,2.828025


In [89]:
df_subset.groupby('number.of.engines').mean()

,event.date,fatal.injuries.perc,nonfatal.injuries.perc,uninjured.perc,aircraft.damage.scale
number.of.engines,,,,,
0,1996.555556,11.111111,6.555556,71.000000,2.000000
1,2009.633703,15.979942,20.591466,60.294021,2.721263
2,2008.841316,19.238027,10.748413,63.454703,2.130698
3,1998.857143,4.642857,5.660714,84.125000,1.553571
4,2007.920792,12.544554,2.336634,61.227723,1.366337
6,2022.000000,0.000000,0.000000,0.000000,3.000000
8,2019.000000,0.000000,0.000000,0.000000,3.000000


In [90]:
df_subset_makes.groupby('make').mean()

,event.date,number.of.engines,fatal.injuries.perc,nonfatal.injuries.perc,uninjured.perc,aircraft.damage.scale
make,,,,,,
aero commander,2007.745283,1.396226,24.528302,25.707547,48.820755,2.726415
aeronca,2009.478261,1.000000,9.273913,26.443478,64.273913,2.904348
air tractor,2012.824444,1.000000,19.333333,23.444444,56.555556,2.828889
airbus,2015.038596,1.680702,4.933333,1.807018,52.347368,0.645614
american champion,2012.229508,1.008197,14.549180,20.762295,64.684426,2.881148
aviat,2013.288973,1.072243,13.498099,22.977186,63.140684,2.749049
beech,2008.653015,1.348692,26.532992,18.350398,53.945961,2.690557
bellanca,2005.654804,1.000000,13.790036,21.530249,64.679715,2.868327
boeing,2012.672168,1.531133,3.267817,4.813953,57.411853,1.069767


In [91]:
df_subset.groupby('engine.type').mean()

,event.date,number.of.engines,fatal.injuries.perc,nonfatal.injuries.perc,uninjured.perc,aircraft.damage.scale
engine.type,,,,,,
Electric,2017.000000,3.600000,0.000000,0.000000,20.000000,2.800000
Geared Turbofan,2019.166667,1.916667,0.000000,0.000000,8.333333,0.083333
N/A,2017.117445,1.157740,26.767568,14.690663,41.529975,2.033661
Reciprocating,2007.583838,1.087539,14.242429,21.151245,64.423789,2.847577
Turbo Fan,2011.055151,2.041623,6.356920,4.629553,75.072841,1.339230
Turbo Jet,2007.196078,1.941176,18.176471,11.202614,66.555556,1.816993
Turbo Prop,2010.593985,1.382707,20.921053,16.172932,60.906767,2.563910
Turbo Shaft,2014.545455,1.272727,9.090909,27.272727,54.545455,2.545455


In [92]:
df_subset.groupby('weather.condition').mean()

,event.date,number.of.engines,fatal.injuries.perc,nonfatal.injuries.perc,uninjured.perc,aircraft.damage.scale
weather.condition,,,,,,
IMC,2005.512858,1.369581,53.273328,14.257899,31.587068,2.380602
N/A,2016.687668,1.254380,27.691712,8.764825,37.307615,1.693396
UNK,2006.752113,1.281690,34.518310,8.605634,42.670423,1.788732
VMC,2008.723413,1.128288,11.822346,21.150787,66.688624,2.802031


In [93]:
df_subset.groupby('aircraft.damage.scale').mean()

,event.date,number.of.engines,fatal.injuries.perc,nonfatal.injuries.perc,uninjured.perc
aircraft.damage.scale,,,,,
0,2013.972059,1.497059,4.222059,4.958088,56.652941
1,2009.415217,1.566304,0.806522,2.508696,72.616304
2,2005.289869,1.188878,70.124960,17.755193,10.536593
3,2009.859056,1.110519,9.165568,21.108674,68.818455


### Business Recommendation 1

In [94]:
df_subset.groupby('engine.type')['aircraft.damage.scale', 'fatal.injuries.perc'].mean()

<ipython-input-94-6ec350043e64>:1: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  df_subset.groupby('engine.type')['aircraft.damage.scale', 'fatal.injuries.perc'].mean()


,aircraft.damage.scale,fatal.injuries.perc
engine.type,,
Electric,2.800000,0.000000
Geared Turbofan,0.083333,0.000000
N/A,2.033661,26.767568
Reciprocating,2.847577,14.242429
Turbo Fan,1.339230,6.356920
Turbo Jet,1.816993,18.176471
Turbo Prop,2.563910,20.921053
Turbo Shaft,2.545455,9.090909


Tableau Visualization 1: Engine Type

Explain why excluded N/A from graph

Recommendation:

### Business Recommendation 2

In [95]:
df_subset.groupby('number.of.engines')['aircraft.damage.scale', 'fatal.injuries.perc'].mean()

<ipython-input-95-9ecaeae76db8>:1: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  df_subset.groupby('number.of.engines')['aircraft.damage.scale', 'fatal.injuries.perc'].mean()


,aircraft.damage.scale,fatal.injuries.perc
number.of.engines,,
0,2.000000,11.111111
1,2.721263,15.979942
2,2.130698,19.238027
3,1.553571,4.642857
4,1.366337,12.544554
6,3.000000,0.000000
8,3.000000,0.000000


Tableau Visualization 2: Number of Engines

Explain why excluded 0 engines

In [ ]:
Show table of frequencies of number of engines

In [99]:
df_subset['number.of.engines'].value_counts(normalize=True)

1    0.850907
2    0.142201
4    0.004144
3    0.002298
0    0.000369
8    0.000041
6    0.000041
Name: number.of.engines, dtype: float64

Recommendaiton:

### Business Recommendation 3

In [96]:
df_subset_makes.groupby('make')['aircraft.damage.scale', 'fatal.injuries.perc'].mean()

<ipython-input-96-269f3ed2dd11>:1: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  df_subset_makes.groupby('make')['aircraft.damage.scale', 'fatal.injuries.perc'].mean()


,aircraft.damage.scale,fatal.injuries.perc
make,,
aero commander,2.726415,24.528302
aeronca,2.904348,9.273913
air tractor,2.828889,19.333333
airbus,0.645614,4.933333
american champion,2.881148,14.549180
aviat,2.749049,13.498099
beech,2.690557,26.532992
bellanca,2.868327,13.790036
boeing,1.069767,3.267817


Tableau Visualization 3: Aircraft Damage

Recommendation:

## Conclusion

Our data concludes that our company's venture into the airline business should involve Airbus or Boeing turbo fan planes.

If our company wishes to start with a private, chartered plane business involving fewer passengers, we recommend purchasing single engine planes. If our company wishes to make a bigger investment into commercial passenger flights, we recommend purchasing dual engine planes. 

### Next Steps

While our results have been very informative to our business venture, there are more data exploration possibilities that could inform other aspects of the business. Although such questions were not within the scope of this project, a future researcher could consider adding back data that we removed from the original, full NTSB dataset in order to draw conclusions that flesh out what a safe and cost-effective aerospace business should look like.

Some examples of questions that could be answered using the available data are: the safetest area of the world to fly, whether private vs. commercial flights are safer, whether certain airports experienced greater percentages of fatalities, which phase of flight tended to be most dangerous, etc.

We have provided the full NTSB dataset, our two cleaned subsets, and our Tableau Dashboard in our repository. We encourage other data scientists to explore the full possibilities of research in this arena!
